In [27]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys, os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))

In [2]:
from PerceptualAudio_Pytorch.models import JNDnet

from speech_distances.datasets import load_dataset
from speech_distances.dpam import MetricDPAM

import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import IPython.display as ipd
from torchvision.datasets import ImageFolder

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

/home/darayavaus/projects/env/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
[NeMo W 2021-05-28 15:24:43 nemo_logging:349] /home/darayavaus/projects/env/lib/python3.7/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but

## Dataset

In [3]:
def pad_collate(batch):
    (wavs, samplerates,  _, transcripts) = zip(*batch)  
    wavs = [torch.squeeze(w) for w in wavs]
    return pad_sequence(wavs, batch_first=True, padding_value=0.0)

In [4]:
dataset = load_dataset('ljspeech')
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=pad_collate, num_workers=4)

In [5]:
iterator = iter(dataloader)
waveforms = next(iterator)
waveforms.shape

torch.Size([1, 212893])

In [6]:
ipd.Audio(waveforms[0].numpy(), rate=22050)

## Metric

In [7]:
random_waveforms = torch.zeros(*waveforms.shape).uniform_(-1,1)
random_waveforms.shape

torch.Size([1, 212893])

In [12]:
metric = MetricDPAM(path='../data/small_ljspeech_wavs', 
                    reference_path='../data/small_ljspeech_wavs', 
                    batch_size=1,
                    device='cpu')
metric.distance(waveforms, waveforms), metric.distance(waveforms, random_waveforms)

(tensor(0.), tensor(3.4152))

In [19]:
metric = MetricDPAM(path='../data/small_ljspeech_wavs', 
                    reference_path='../data/small_ljspeech_wavs', 
                    batch_size=10,
                    device='cpu')
metric.distance(waveforms, waveforms), metric.distance(waveforms, random_waveforms)

(tensor(0.), tensor(3.4152))

In [36]:
metric = MetricDPAM(path='./val_dataset', 
                    reference_path='./data/out/hifigan_v1/', 
                    batch_size=1,
                    device='cpu')
metric.calculate_metric()

tensor(2.0851)

In [37]:
metric = MetricDPAM(path='./val_dataset', 
                    reference_path='./data/out/hifigan_v2/', 
                    batch_size=1,
                    device='cpu')
metric.calculate_metric()

tensor(2.0737)

In [38]:
metric = MetricDPAM(path='./val_dataset', 
                    reference_path='./data/out/hifigan_v3/', 
                    batch_size=1,
                    device='cpu')
metric.calculate_metric()

tensor(2.0726)